In [6]:
from pathlib import Path
from collections import Counter
from datetime import date

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from skl

In [3]:
DATA_PATH = Path.cwd().parent.joinpath('data', 'raw')

In [61]:
consumer_staples = pd.read_csv(DATA_PATH.joinpath('sp500_consumer_staples.csv'))
consumer_staples = consumer_staples.iloc[::-1]
prices = pd.read_csv(DATA_PATH.joinpath('month_end_prices.csv'))
t_bill = pd.read_csv(DATA_PATH.joinpath('3_month_t_bill.csv'))
t_bill = t_bill[::-1]

In [62]:
consumer_staples.head()

,Date,Price,Open,High,Low,Vol.,Change %
255,Jan 00,193.87,193.87,193.87,193.87,-,-6.82%
254,Feb 00,174.01,174.01,174.01,174.01,-,-10.24%
253,Mar 00,171.55,171.55,171.55,171.55,-,-1.41%
252,Apr 00,175.73,175.73,175.73,175.73,-,2.44%
251,May 00,192.32,192.32,192.32,192.32,-,9.44%


In [63]:
prices.head()

,gvkey,iid,datadate,tic,cusip,conm,prccm,exchg,secstat,tpci,cik,fic
0,1213,01,20000131,ABF,009269101,AIRBORNE INC,19.5625,11,I,0,3000.0,USA
1,1213,01,20000229,ABF,009269101,AIRBORNE INC,18.5000,11,I,0,3000.0,USA
2,1213,01,20000331,ABF,009269101,AIRBORNE INC,24.0000,11,I,0,3000.0,USA
3,1213,01,20000430,ABF,009269101,AIRBORNE INC,21.4375,11,I,0,3000.0,USA
4,1213,01,20000531,ABF,009269101,AIRBORNE INC,20.1875,11,I,0,3000.0,USA


In [64]:
t_bill.head()

,Date,Price,Open,High,Low,Change %
255,Jan 00,5.692,5.692,5.692,5.692,9.31%
254,Feb 00,5.792,5.792,5.792,5.792,1.76%
253,Mar 00,5.860,5.860,5.860,5.860,1.17%
252,Apr 00,5.813,5.813,5.813,5.813,-0.80%
251,May 00,5.613,5.613,5.613,5.613,-3.44%


In [66]:
def get_bad_data(prices):
    bad_data = list()
    for ticker in prices.tic.unique():
        subset = prices[prices.tic==ticker]
        current = 'na' if np.isnan(subset.prccm.iloc[0]) else 'val'
        for v in subset.prccm[1:]:
            if not np.isnan(v):
                current = 'val'
            elif current == 'val' and np.isnan(v):
                bad_data.append((ticker, subset.conm.iloc[0]))
                continue
    return Counter(bad_data)

In [46]:
get_bad_data(prices)

Counter({('ABF', 'AIRBORNE INC'): 1,
         ('HRB', 'BLOCK H & R INC'): 1,
         ('BF.B', 'BROWN FORMAN CORP'): 1,
         ('BF.A', 'BROWN FORMAN CORP'): 1,
         ('CDSCY', 'CADBURY PLC'): 1,
         ('0491B', 'DOLE FOOD CO INC'): 1,
         ('KO', 'COCA-COLA CO'): 1,
         ('TAP.Z.', 'MOLSON COORS BEVERAGE CO'): 80,
         ('TAP.Z', 'MOLSON COORS BEVERAGE CO'): 81,
         ('DCNAQ', 'DANA INC'): 1,
         ('DG.1', 'DOLLAR GENERAL CORP'): 1,
         ('DRYR', "DREYER'S GRAND ICE CRM HLDGS"): 1,
         ('GM.1', 'GENERAL MOTORS CO'): 33,
         ('XGM', 'GENERAL MOTORS CO'): 1,
         ('HGM', 'GENERAL MOTORS CO'): 1,
         ('BGM.1', 'GENERAL MOTORS CO'): 1,
         ('GMS.1', 'GENERAL MOTORS CO'): 1,
         ('GPC', 'GENUINE PARTS CO'): 1,
         ('4741B', 'HERSHEY CO'): 127,
         ('HOVNP', 'HOVNANIAN ENTRPRS INC  -CL A'): 1,
         ('PAS', 'PEPSIAMERICAS INC'): 1,
         ('KWD', 'KELLWOOD CO'): 1,
         ('LDG', 'LONGS DRUG STORES CORP'): 1,
     

In [67]:
prices['chng'] = prices.prccm.pct_change()

In [68]:
prices.head()

,gvkey,iid,datadate,tic,cusip,conm,prccm,exchg,secstat,tpci,cik,fic,chng
0,1213,01,20000131,ABF,009269101,AIRBORNE INC,19.5625,11,I,0,3000.0,USA,NaN
1,1213,01,20000229,ABF,009269101,AIRBORNE INC,18.5000,11,I,0,3000.0,USA,-0.054313
2,1213,01,20000331,ABF,009269101,AIRBORNE INC,24.0000,11,I,0,3000.0,USA,0.297297
3,1213,01,20000430,ABF,009269101,AIRBORNE INC,21.4375,11,I,0,3000.0,USA,-0.106771
4,1213,01,20000531,ABF,009269101,AIRBORNE INC,20.1875,11,I,0,3000.0,USA,-0.058309


In [69]:
consumer_staples['chng'] = consumer_staples.Price.pct_change()

In [70]:
consumer_staples.head()

,Date,Price,Open,High,Low,Vol.,Change %,chng
255,Jan 00,193.87,193.87,193.87,193.87,-,-6.82%,NaN
254,Feb 00,174.01,174.01,174.01,174.01,-,-10.24%,-0.102440
253,Mar 00,171.55,171.55,171.55,171.55,-,-1.41%,-0.014137
252,Apr 00,175.73,175.73,175.73,175.73,-,2.44%,0.024366
251,May 00,192.32,192.32,192.32,192.32,-,9.44%,0.094406


In [71]:
consumer_staples['excess'] = consumer_staples.chng - t_bill.Price/100

In [72]:
consumer_staples.excess.mean()

-0.009783040046646931